# Filter EHR patients

Filter out:
- patients with missmatch in patient_id (from registry) and eds_final_patient_id (from EHR extraction)
    - Goal: avoid using data from another patient that was mistakenly selected during EHR extraction
    - Exception: patient with manually completed EDS or ID (patient_id_manual, eds_manual)

__This code is now in:__ `patient_selection/filter_ehr_patients.py`


In [ ]:
import pandas as pd
import os

In [ ]:
data_path = '/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/errors_Extraction_20220815/eds_j1.csv'

In [ ]:
data_df = pd.read_csv(data_path, delimiter=';', encoding='utf-8',
                         dtype=str)

In [ ]:
# selected rows
data_df[(data_df['patient_id'] != data_df['eds_final_patient_id'])
        & ((data_df.eds_manual.isna()) | (data_df.match_by != '0 = eds manual'))
]

In [ ]:
data_df[(~data_df.patient_id_manual.isna() & (data_df.match_by != '0 = eds manual'))]

In [ ]:
# drop selected rows
filtered_data_df = data_df.drop(data_df[(data_df['patient_id'] != data_df['eds_final_patient_id'])
        & ((data_df.eds_manual.isna()) | (data_df.match_by != '0 = eds manual'))].index)

## Testing final function

In [ ]:
data_path = '/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction20220629'
lab_file_start = 'labo'

In [ ]:
lab_files = [pd.read_csv(os.path.join(data_path, f), delimiter=';', encoding='utf-8', dtype=str)
             for f in os.listdir(data_path)
             if f.startswith(lab_file_start)]

In [ ]:
from preprocessing.utils import create_ehr_case_identification_column

lab_df = pd.concat(lab_files, ignore_index=True)
lab_df['case_admission_id'] = create_ehr_case_identification_column(lab_df)

In [ ]:
from preprocessing.patient_selection.filter_ehr_patients import filter_ehr_patients

filtered_df = filter_ehr_patients(lab_df)

In [ ]:
len(filtered_df.case_admission_id.unique()), len(lab_df.case_admission_id.unique())

In [ ]:
lab_df[(lab_df['patient_id'] != lab_df['eds_final_patient_id'])
        & ((lab_df.eds_manual.isna()) | (lab_df.match_by != '0 = eds manual'))
]

In [ ]:
filtered_df[(filtered_df['patient_id'] != filtered_df['eds_final_patient_id'])
        & ((filtered_df.eds_manual.isna()) | (filtered_df.match_by != '0 = eds manual'))
]

Function works as expected.